In [ ]:
import sagemaker
import boto3
from sagemaker.model import Model
from sagemaker.model import FrameworkModel
#from sagemaker.pipeline_model import PipelineModel
from sagemaker.pipeline import PipelineModel
from sagemaker import get_execution_role

from time import gmtime, strftime

from sagemaker.predictor import json_serializer, json_deserializer, Predictor
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer

In [ ]:
sagemaker_session = sagemaker.Session()
role = get_execution_role()

In [ ]:
model_a_name = 'mock-model-a-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
model_b_name = 'mock-model-b-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
model_pipeline_name = 'mock-model-pipeline-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())

In [ ]:
ecr_repository_name = "mmkv"
account_id = role.split(':')[4]
region = boto3.Session().region_name

container_image_uri = '{0}.dkr.ecr.{1}.amazonaws.com/{2}:latest'.format(account_id, region, ecr_repository_name)
print(container_image_uri)

 --- --- Version 1 - working example --- ---

In [ ]:
model_a = Model(name = model_a_name,   
                       image_uri = container_image_uri,
                       role=role,
                       env = {
                           'SAGEMAKER_USE_NGINX': 'true',
                           'SAGEMAKER_MODEL_SERVER_WORKERS': '1',
                           'SAGEMAKER_MODEL_SERVER_TIMEOUT': '300'
                       },
                       #code_location=s3_code_location,
                       #sagemaker_session=sagemaker_session #comment this line for local mode.
                      )

In [ ]:
model_b = Model(name = model_b_name,   
                       image_uri = container_image_uri,
                       role=role,
                       env = {
                           'SAGEMAKER_USE_NGINX': 'true',
                           'SAGEMAKER_MODEL_SERVER_WORKERS': '1',
                           'SAGEMAKER_MODEL_SERVER_TIMEOUT': '300'
                       },
                       #code_location=s3_code_location,
                       #sagemaker_session=sagemaker_session #comment this line for local mode.
                      )

In [ ]:
endpoint_name = 'mock-models-ep-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print(endpoint_name)

In [ ]:
# Create SageMaker Inference Pipeline, serial mode

pipeline_model = PipelineModel(name=model_pipeline_name,
                               role=role,
                               models=[
                                    model_a,
                                    model_b
                               ])

In [ ]:
# uncomment below if desire to deploy a single model

# pred = model_a.deploy(initial_instance_count=1,
#                     #instance_type='local',
#                     instance_type="ml.t2.medium",
#                     endpoint_name=endpoint_name)

In [ ]:
pred = pipeline_model.deploy(initial_instance_count=1,
                    #instance_type="ml.m5.large",
                    instance_type= "ml.c4.xlarge",
                    endpoint_name=endpoint_name)

In [ ]:
predictor = Predictor(endpoint_name="mock-models-ep-2021-10-12-23-15-19",
#                     endpoint_name=endpoint_name,
                      sagemaker_session=sagemaker.Session(),
                      serializer=JSONSerializer(),
                      deserializer=JSONDeserializer(),
#                       content_type='application/json',
#                       accept='application/json'
                     )

In [ ]:
payload = {"key": "sme", "value": 10}

In [ ]:
print(predictor.predict(payload))

 --- --- Version 2 - working example --- ---

In [ ]:
predictor.delete_endpoint()